## Download **Dataset** and **InstructABSA**

In [1]:
!git clone https://github.com/kevinscaria/InstructABSA.git
!gdown https://drive.google.com/u/0/uc?id=1w55HemhcOOVa4keKWKqwnTFIU6jGyDUi&export=download
!unzip UIT-ViSFD.zip -d InstructABSA/Dataset/UIT-ViSFD/

!mkdir "/content/InstructABSA/Dataset/UIT-ViSFD/Train"
!mkdir "/content/InstructABSA/Dataset/UIT-ViSFD/Test"
!mkdir "/content/InstructABSA/Dataset/UIT-ViSFD/Validation"

Cloning into 'InstructABSA'...
remote: Enumerating objects: 542, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 542 (delta 112), reused 121 (delta 62), pack-reused 338
Receiving objects: 100% (542/542), 1.58 MiB | 10.43 MiB/s, done.
Resolving deltas: 100% (309/309), done.
Downloading...
From: https://drive.google.com/u/0/uc?id=1w55HemhcOOVa4keKWKqwnTFIU6jGyDUi
To: /content/UIT-ViSFD.zip
100% 939k/939k [00:00<00:00, 182MB/s]
Archive:  UIT-ViSFD.zip
  inflating: InstructABSA/Dataset/UIT-ViSFD/Dev.csv  
  inflating: InstructABSA/Dataset/UIT-ViSFD/Test.csv  
  inflating: InstructABSA/Dataset/UIT-ViSFD/Train.csv  


## Libraries

In [2]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [3]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 116.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.7 MB/s eta 0:00:00


In [4]:
import os
import torch

if IN_COLAB:
    root_path = '/content/InstructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [5]:
!pip install -r requirements.txt
!pip install transformers[torch]
!pip install accelerate -U
!pip install py_vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 21.3 MB/s eta 0:00:00
  Created wheel for py_vncorenlp: filename=py_vncorenlp-0.1.4-py3-none-any.whl size=4305 sha256=aeea2f225d4d4b4a89f16b9a8299f5bc8a659d0a6d03c70adfb984355a40a5f4
  Stored in directory: /root/.cache/pip/wheels/d5/d9/bf/62632cdb007c702a0664091e92a0bb1f18a2fcecbe962d9827
Successfully built py_vncorenlp


In [6]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import string

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

# import py_vncorenlp
# py_vncorenlp.download_model(save_dir='/content')

# rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/content')

## Create Custom Dataset

In [ ]:
train_ds = pd.read_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Train.csv')
test_ds = pd.read_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Test.csv')
val_ds = pd.read_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Dev.csv')

In [ ]:
def pre_proc(dataset):
  dataset = dataset.drop(['n_star', 'date_time'], axis=1).set_index('index').rename(columns={'comment': 'raw_text', 'label': 'aspectTerms'})

  #Word segment for Vietnamese sentences
  def word_segmentation(data):
    data = ''.join([x for x in data if x not in string.punctuation])
    segments = rdrsegmenter.word_segment(data)
    try:
      return segments[0]
    except:
      return segments
  dataset.raw_text = dataset.raw_text.map(word_segmentation)

  #Custom labels to specific format
  def customLabel(data):
    tmp = [datum.strip('{}').split('#') for datum in data.split(';')][:-1]
    tmp = [datum+['None'] if datum == ['OTHERS'] else datum for datum in tmp]
    return [{'term': raw_text, 'polarity': aspectTerm.lower()} for raw_text, aspectTerm in tmp if tmp]

  dataset.aspectTerms = dataset.aspectTerms.map(customLabel)

  return dataset

In [ ]:
train_ds = pre_proc(train_ds)
test_ds = pre_proc(test_ds)
val_ds = pre_proc(val_ds)

In [ ]:
train_ds.to_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Train/Train.csv')
test_ds.to_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Test/Test.csv')
val_ds.to_csv('/content/InstructABSA/Dataset/UIT-ViSFD/Validation/Val.csv')

## Training

In [ ]:
task_name = 'joint_task'
experiment_name = 'ViSFD'
model_checkpoint = 'kevinscaria/joint_tk-instruct-base-def-pos-combined'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  ViSFD
Model output path:  ./Models/joint_task/allenaitk-instruct-base-def-pos-ViSFD


In [ ]:
# Load the data
id_train_file_path = '/content/InstructABSA/Dataset/UIT-ViSFD/Train/Train.csv'
id_test_file_path = '/content/InstructABSA/Dataset/UIT-ViSFD/Test/Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [ ]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir': model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':8,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'epoch',
    'load_best_model_at_end':True,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Map:   0%|          | 0/7786 [00:00<?, ? examples/s]

Map:   0%|          | 0/2224 [00:00<?, ? examples/s]

In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,0.151400,0.345481
2,0.192200,0.217271
3,0.170700,0.226521
4,0.154900,0.213665
5,0.149400,0.219803
6,0.150000,0.210835
7,0.155400,0.205061
8,0.160000,0.204835


## Inference

In [ ]:
# Load the data
id_train_file_path = '/content/InstructABSA/Dataset/UIT-ViSFD/Train/Train.csv'
id_test_file_path = '/content/InstructABSA/Dataset/UIT-ViSFD/Test/Test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set1()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_joint_task_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_joint_task_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.joint['bos_instruct1'], instruct_handler.joint['eos_instruct'])

In [ ]:
# # Model inference - Loading from Checkpoint
# t5_exp = T5Generator(model_out_path)

# # Tokenize Datasets
# id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Model loaded to:  cuda


100%|██████████| 487/487 [10:50<00:00,  1.34s/it]


Model loaded to:  cuda


100%|██████████| 139/139 [03:03<00:00,  1.32s/it]


In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

Train Precision:  0.7935483870967742
Train Recall:  0.7890166627552218
Train F1:  0.7912760365590553
Test Precision:  0.7179163156416737
Test Recall:  0.7067035245335176
Test F1:  0.7122657936894895
